In [1]:
import numpy as np

In [2]:
a = np.array([0,1,0,0,198, 0,0,0,12,2,34,5,0,0,0])
a

array([  0,   1,   0,   0, 198,   0,   0,   0,  12,   2,  34,   5,   0,
         0,   0])

In [8]:
b = a.nonzero()[0]
w = (b[-1]-b[0])//(len(b)) +1
w

2

In [9]:
b

array([ 1,  4,  8,  9, 10, 11])

In [11]:
c = np.array([0])
c.nonzero()[0]

array([], dtype=int64)

In [12]:
3*[1]

[1, 1, 1]

In [13]:
int(1.3)

1

In [14]:
np.concatenate((c,a), axis=0)

array([  0,   0,   1,   0,   0, 198,   0,   0,   0,  12,   2,  34,   5,
         0,   0,   0])

In [15]:
import pandas

In [ ]:
pandas.DataFrame()